In [1]:
from pyomo.environ import (Constraint,
                           Var,
                           ConcreteModel,
                           Expression,
                           Param,
                           Objective,
                           SolverFactory,
                           TransformationFactory,
                           value)

from pyomo.network import Arc, SequentialDecomposition

from idaes.core.util.initialization import propagate_state

# Import plotting functions
import matplotlib.pyplot as plt

# Import numpy library 
import numpy as np

# Import the main FlowsheetBlock from IDAES. The flowsheet block will contain the unit model
from idaes.core import FlowsheetBlock

import idaes.logger as idaeslog
from pyomo.opt import TerminationCondition, SolverStatus

# Import the degrees_of_freedom function from the idaes.core.util.model_statistics package
# DOF = Number of Model Variables - Number of Model Constraints
from idaes.core.util.model_statistics import degrees_of_freedom

# Import the Generic Parameter Block
from idaes.generic_models.properties.core.generic.generic_property import (
        GenericParameterBlock)

# Import unit models from the model library
from idaes.generic_models.unit_models import Flash

# Import R32, R125, IL info file
from HFCs_bmimPF6_PR import configuration

In [2]:
m1 = ConcreteModel()

m1.fs = FlowsheetBlock(default={"dynamic": False})

m1.fs.properties = GenericParameterBlock(default=configuration)

m1.fs.F101 = Flash(default={"property_package": m1.fs.properties,
                           "has_heat_transfer": True,
                           "has_pressure_change": True})

m2 = ConcreteModel()

m2.fs = FlowsheetBlock(default={"dynamic": False})

m2.fs.properties = GenericParameterBlock(default=configuration)

m2.fs.F101 = Flash(default={"property_package": m2.fs.properties,
                           "has_heat_transfer": True,
                           "has_pressure_change": True})

m3 = ConcreteModel()

m3.fs = FlowsheetBlock(default={"dynamic": False})

m3.fs.properties = GenericParameterBlock(default=configuration)

m3.fs.F101 = Flash(default={"property_package": m3.fs.properties,
                           "has_heat_transfer": True,
                           "has_pressure_change": True})

In [3]:
m1.fs.properties.PR_kappa_A['R32', 'R125'].fix(-0.09826)
m1.fs.properties.PR_kappa_A['R125', 'R32'].fix(-0.05798)
m1.fs.properties.PR_kappa_A['R32', 'bmimPF6'].fix(-0.06350)
m1.fs.properties.PR_kappa_A['bmimPF6', 'R32'].fix(-0.40220)
m1.fs.properties.PR_kappa_A['bmimPF6', 'R125'].fix(1.58822)
m1.fs.properties.PR_kappa_A['R125', 'bmimPF6'].fix(-0.00107)

m2.fs.properties.PR_kappa_A['R32', 'R125'].fix(0.009)
m2.fs.properties.PR_kappa_A['R125', 'R32'].fix(0.007)
m2.fs.properties.PR_kappa_A['R32', 'bmimPF6'].fix(-0.05450)
m2.fs.properties.PR_kappa_A['bmimPF6', 'R32'].fix(-0.10943)
m2.fs.properties.PR_kappa_A['bmimPF6', 'R125'].fix(0.28636)
m2.fs.properties.PR_kappa_A['R125', 'bmimPF6'].fix(-0.00356)

m3.fs.properties.PR_kappa_A['R32', 'R125'].fix(0.39255)
m3.fs.properties.PR_kappa_A['R125', 'R32'].fix(-0.375437)
m3.fs.properties.PR_kappa_A['R32', 'bmimPF6'].fix(0.06399)
m3.fs.properties.PR_kappa_A['bmimPF6', 'R32'].fix(0.63257)
m3.fs.properties.PR_kappa_A['bmimPF6', 'R125'].fix(-1.195988)
m3.fs.properties.PR_kappa_A['R125', 'bmimPF6'].fix(-0.088142)
m3.fs.properties.PR_kappa_B['R32', 'R125'].fix(-0.389117)
m3.fs.properties.PR_kappa_B['R125', 'R32'].fix(0.3892)
m3.fs.properties.PR_kappa_B['R32', 'bmimPF6'].fix(-0.10862)
m3.fs.properties.PR_kappa_B['bmimPF6', 'R32'].fix(-0.69149)
m3.fs.properties.PR_kappa_B['bmimPF6', 'R125'].fix(1.5687447)
m3.fs.properties.PR_kappa_B['R125', 'bmimPF6'].fix(0.084894)

In [4]:
solver = SolverFactory('ipopt')
solver.options = {'tol': 1e-6}

In [5]:
# Set pressures to loop over
R125_z = [0.25]

# Make outlet arrays
IL_x = np.zeros(len(R125_z))
IL_y = np.zeros(len(R125_z))
R32_x = np.zeros(len(R125_z))
R32_y = np.zeros(len(R125_z))
R125_x = np.zeros(len(R125_z))
R125_y = np.zeros(len(R125_z))

IL_x2 = np.zeros(len(R125_z))
IL_y2 = np.zeros(len(R125_z))
R32_x2 = np.zeros(len(R125_z))
R32_y2 = np.zeros(len(R125_z))
R125_x2 = np.zeros(len(R125_z))
R125_y2 = np.zeros(len(R125_z))

IL_x3 = np.zeros(len(R125_z))
IL_y3 = np.zeros(len(R125_z))
R32_x3 = np.zeros(len(R125_z))
R32_y3 = np.zeros(len(R125_z))
R125_x3 = np.zeros(len(R125_z))
R125_y3 = np.zeros(len(R125_z))
L3 = np.zeros(len(R125_z))
V3 = np.zeros(len(R125_z))

m1.fs.F101.inlet.flow_mol.fix(10)
m1.fs.F101.inlet.temperature.fix(298)
m1.fs.F101.inlet.mole_frac_comp[0, "bmimPF6"].fix(0.9) 
m1.fs.F101.inlet.pressure.fix(100000)

m2.fs.F101.inlet.flow_mol.fix(10)
m2.fs.F101.inlet.temperature.fix(298)
m2.fs.F101.inlet.mole_frac_comp[0, "bmimPF6"].fix(0.9) 
m2.fs.F101.inlet.pressure.fix(100000)

m3.fs.F101.inlet.flow_mol.fix(10)
m3.fs.F101.inlet.temperature.fix(298)
m3.fs.F101.inlet.mole_frac_comp[0, "bmimPF6"].fix(0.9) 
m3.fs.F101.inlet.pressure.fix(100000)

m1.fs.F101.vap_outlet.pressure.fix(100000)
m1.fs.F101.vap_outlet.temperature.fix(298.15)
m2.fs.F101.vap_outlet.pressure.fix(100000)
m2.fs.F101.vap_outlet.temperature.fix(298.15)
m3.fs.F101.vap_outlet.pressure.fix(100000)
m3.fs.F101.vap_outlet.temperature.fix(298.15)

m1.fs.liq = Param(mutable=True, default=0.3)
m1.fs.liquid =  Constraint(expr=m1.fs.F101.vap_outlet.mole_frac_comp[0, "R32"] == m1.fs.liq)
m1.fs.liquid2 =  Constraint(expr=m1.fs.F101.vap_outlet.mole_frac_comp[0, "R125"] == 1-m1.fs.liq)

m2.fs.liq = Param(mutable=True, default=0.3)
m2.fs.liquid =  Constraint(expr=m2.fs.F101.vap_outlet.mole_frac_comp[0, "R32"] == m1.fs.liq)
m2.fs.liquid2 =  Constraint(expr=m2.fs.F101.vap_outlet.mole_frac_comp[0, "R125"] == 1-m1.fs.liq)

m3.fs.liq = Param(mutable=True, default=0.3)
m3.fs.liquid =  Constraint(expr=m3.fs.F101.vap_outlet.mole_frac_comp[0, "R32"] == m1.fs.liq)
m3.fs.liquid2 =  Constraint(expr=m3.fs.F101.vap_outlet.mole_frac_comp[0, "R125"] == 1-m1.fs.liq)


for i in range(len(R125_z)):
    
    m1.fs.F101.inlet.flow_mol.fix(10)
    m1.fs.F101.inlet.temperature.fix(298)
    m1.fs.F101.inlet.mole_frac_comp[0, "bmimPF6"].fix(0.5) 
    m1.fs.F101.inlet.pressure.fix(100000)

    m2.fs.F101.inlet.flow_mol.fix(10)
    m2.fs.F101.inlet.temperature.fix(298)
    m2.fs.F101.inlet.mole_frac_comp[0, "bmimPF6"].fix(0.5) 
    m2.fs.F101.inlet.pressure.fix(100000)

    m3.fs.F101.inlet.flow_mol.fix(10)
    m3.fs.F101.inlet.temperature.fix(298)
    m3.fs.F101.inlet.mole_frac_comp[0, "bmimPF6"].fix(0.5) 
    m3.fs.F101.inlet.pressure.fix(100000)

    m1.fs.F101.vap_outlet.pressure.fix(100000)
    m1.fs.F101.vap_outlet.temperature.fix(298.15)
    m2.fs.F101.vap_outlet.pressure.fix(100000)
    m2.fs.F101.vap_outlet.temperature.fix(298.15)
    m3.fs.F101.vap_outlet.pressure.fix(100000)
    m3.fs.F101.vap_outlet.temperature.fix(298.15)   
    
    m1.fs.liq = R125_z[i]
    m2.fs.liq = R125_z[i]
    m3.fs.liq = R125_z[i]

    m1.fs.F101.initialize(outlvl=idaeslog.CRITICAL)              
    m2.fs.F101.initialize(outlvl=idaeslog.CRITICAL)
    m3.fs.F101.initialize(outlvl=idaeslog.CRITICAL)
    
    # solve the model
    status1 = solver.solve(m1, tee = False)
    status2 = solver.solve(m2, tee = False)
    status3 = solver.solve(m3, tee = False)
        
    # If solution is optimal store the concentration, and calculated temperatures in the created arrays
    if (status1.solver.status == SolverStatus.ok) and (status1.solver.termination_condition == TerminationCondition.optimal):

        IL_x[i] = value(m1.fs.F101.liq_outlet.mole_frac_comp[0, "bmimPF6"])
        IL_y[i] = value(m1.fs.F101.vap_outlet.mole_frac_comp[0, "bmimPF6"])
        R32_x[i] = value(m1.fs.F101.liq_outlet.mole_frac_comp[0, "R32"]) 
        R32_y[i] = value(m1.fs.F101.vap_outlet.mole_frac_comp[0, "R32"])
        R125_x[i] = value(m1.fs.F101.liq_outlet.mole_frac_comp[0, "R125"])
        R125_y[i] = value(m1.fs.F101.vap_outlet.mole_frac_comp[0, "R125"])

    if (status2.solver.status == SolverStatus.ok) and (status2.solver.termination_condition == TerminationCondition.optimal):
        IL_x2[i] = value(m2.fs.F101.liq_outlet.mole_frac_comp[0, "bmimPF6"])
        IL_y2[i] = value(m2.fs.F101.vap_outlet.mole_frac_comp[0, "bmimPF6"])
        R32_x2[i] = value(m2.fs.F101.liq_outlet.mole_frac_comp[0, "R32"]) 
        R32_y2[i] = value(m2.fs.F101.vap_outlet.mole_frac_comp[0, "R32"])
        R125_x2[i] = value(m2.fs.F101.liq_outlet.mole_frac_comp[0, "R125"])
        R125_y2[i] = value(m2.fs.F101.vap_outlet.mole_frac_comp[0, "R125"])

    if (status3.solver.status == SolverStatus.ok) and (status3.solver.termination_condition == TerminationCondition.optimal):  

        IL_x3[i] = value(m3.fs.F101.liq_outlet.mole_frac_comp[0, "bmimPF6"])
        IL_y3[i] = value(m3.fs.F101.vap_outlet.mole_frac_comp[0, "bmimPF6"])
        R32_x3[i] = value(m3.fs.F101.liq_outlet.mole_frac_comp[0, "R32"]) 
        R32_y3[i] = value(m3.fs.F101.vap_outlet.mole_frac_comp[0, "R32"])
        R125_x3[i] = value(m3.fs.F101.liq_outlet.mole_frac_comp[0, "R125"])
        R125_y3[i] = value(m3.fs.F101.vap_outlet.mole_frac_comp[0, "R125"])

        #Change this to feasible
        print("Running R125:",R125_x[i])

    else:
        print('Infeasible.')

Running R125: 0.019895151787794094


In [6]:
# m1.fs.F101.report()

In [7]:
print(R32_x)
print(R32_y)
print(R125_x)
print(R125_y)
print(IL_x)
print(IL_y)
print(R32_x2)
print(R32_y2)
print(R125_x2)
print(R125_y2)
print(IL_x2)
print(IL_y2)
print(R32_x3)
print(R32_y3)
print(R125_x3)
print(R125_y3)
print(IL_x3)
print(IL_y3)

[0.02416096]
[0.25]
[0.01989515]
[0.75]
[0.95594389]
[1.e-08]
[0.02153825]
[0.25]
[0.01971671]
[0.75]
[0.95874504]
[1.e-08]
[0.01960772]
[0.25]
[0.01973398]
[0.75]
[0.9606583]
[1.e-08]


In [8]:
import pandas as pd

df = pd.DataFrame(columns=['IL_x', 'R32_x', 'R125_x','IL_y', 'R32_y', 'R125_y',
                          'IL_x2', 'R32_x2', 'R125_x2','IL_y2', 'R32_y2', 'R125_y2',
                          'IL_x3', 'R32_x3', 'R125_x3','IL_y3', 'R32_y3', 'R125_y3'])
# df.to_csv('P_cons_125.csv')
# df.to_csv('P_lin_125.csv')
df['IL_x'] = IL_x
df['R32_x'] = R32_x
df['R125_x'] = R125_x
df['IL_y'] = IL_y
df['R32_y'] = R32_y
df['R125_y'] = R125_y
df['IL_x2'] = IL_x2
df['R32_x2'] = R32_x2
df['R125_x2'] = R125_x2
df['IL_y2'] = IL_y2
df['R32_y2'] = R32_y2
df['R125_y2'] = R125_y2
df['IL_x3'] = IL_x3
df['R32_x3'] = R32_x3
df['R125_x3'] = R125_x3
df['IL_y3'] = IL_y3
df['R32_y3'] = R32_y3
df['R125_y3'] = R125_y3

In [9]:
# df.to_csv('ku_100.csv')